# Hipercor

#### Carrefour no da resultados, así que necesitamos un plan H


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
import re

Repetimos el proceso del scraping de día para carrefour

In [2]:
df = pd.read_csv('/Users/Natalio/Desktop/Cibeles/Ironhack/Proyecto_Final/Recursos/en.openfoodfacts.org.products.csv', sep='\t')
codes = df['code'].to_list()
barcodes = [i for i in codes if len(str(i)) == 13]

/Users/Natalio/miniconda3/envs/pfironhack_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,24,25,26,28,44,49) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
starttime = datetime.datetime.now()
dfprice = []
dfpricekg = []
for barcode in barcodes:
    try:
        url = 'https://www.hipercor.es/supermercado/buscar/?term=' + barcode
        html = requests.get(url).content
        bs = BeautifulSoup(html, 'lxml')

        try:
            searchingprice = bs.find_all('div', {'class':'prices-price _current'})
            price = str(re.findall('.+(\d+.+\,.+\d+).+\€', str(searchingprice))[0])
            price = price.replace('</span>', '')
            price = price.replace('<span>', '')
            dfprice.append(price)


            searchingpricerperkg = bs.find_all('div', {'class':'prices-price _pum'})
            pricekg = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingpricerperkg))[0])
            dfpricekg.append(pricekg)
    
        except:
            pass
    except:
        pass

    
result = list(zip(barcodes, dfprice, dfpricekg))

df = pd.DataFrame(result, columns = ['code','price_hipercor','pricekg_hipercor'])
df.to_csv('/Users/Natalio/Desktop/Cibeles/Ironhack/Proyecto_Final/Recursos/hipercor.csv', sep='\t')
endtime = datetime.datetime.now()
duration = endtime - starttime
print(f'El proceso finaliza {endtime}.\n La duración total es: {duration}')


['2,09']
<class 'str'>
['4,64']


In [ ]:
def scrapinginfo(barcodes):
    dfprice = []
    dfpricekg = []
    for barcode in barcodes:
        url = 'https://www.dia.es/compra-online/search?text=' + barcode + '&x=0&y=0'
        html = requests.get(url).content
        bs = BeautifulSoup(html, 'lxml')

        try:
            searchingprice = bs.find_all('p', {'class':'price'})
            price = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingprice))[0])
            dfprice.append(price)

            searchingpricerperkg = bs.find_all('p', {'class':'pricePerKilogram'})
            pricekg = str(re.findall('.*(\d+\,\d+)\s\€', str(searchingpricerperkg))[0])
            dfpricekg.append(pricekg)
    
        except:
            dfprice.append('no data')
            dfpricekg.append('no data')
        
    result = list(zip(barcodes, dfprice, dfpricekg))

    df = pd.DataFrame(result, columns = ['code','price','pricekg'])
    return df

scrapinginfo(barcodes)